In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

## Load data function

In [2]:
def read_csv_file(stock, date):
    year = datetime.strptime(date, "%Y%m%d").year

    # Define possible base directories
    base_dirs = [
        f"{stock}/{year}/{year}_new/",
        f"{stock}/{year}/",
        f"{stock}/{year}_new/"
    ]

    filename_pattern = f"{stock}_{date}_*_Intraday_Preprocessed.csv"

    for base_dir in base_dirs:
        search_pattern = os.path.join(base_dir, filename_pattern)
        matching_files = glob.glob(search_pattern)

        if matching_files:
            # Load the first matching file
            return pd.read_csv(matching_files[0])

    print("File not found in any of the expected locations:")
    for base_dir in base_dirs:
        print(f"  {os.path.join(base_dir, filename_pattern)}")

    return None

In [ ]:
def build_yearly_frames(stock):
    """
    Builds 12 expiry-cycle frames from intraday option data files using pandas.

    Filters out rows where:
    - price_problem == True
    - is_tradable == False

    Returns:
        A list of pandas DataFrames.
    """
    base_dirs = [
        f"{stock}/2024/2024_new/",
        f"{stock}/2024/",
        f"{stock}/2024_new/"
    ]

    file_paths = []
    seen_files = set()

    # Search in all directories
    for base_path in base_dirs:
        pattern = os.path.join(base_path, f"{stock}_*_Intraday_Preprocessed.csv")
        for path in glob.glob(pattern):
            fname = os.path.basename(path)
            if fname not in seen_files:
                file_paths.append(path)
                seen_files.add(fname)

    file_paths.sort()

    # Extract date pairs (start_date, expiry_date)
    date_pairs = []
    for path in file_paths:
        fname = os.path.basename(path)
        try:
            parts = fname.split('_')
            start_date = parts[1]
            expiry_date = parts[2]
            date_pairs.append((start_date, expiry_date, path))
        except IndexError:
            continue

    date_pairs.sort(key=lambda x: x[0])

    frames = []
    used_indices = set()
    current_index = 0

    while len(frames) < 12 and current_index < len(date_pairs):
        frame_paths = []
        start_date, expiry_date, _ = date_pairs[current_index]
        frame_start = start_date
        frame_expiry = expiry_date

        # Collect paths for current expiry cycle
        for i in range(current_index, len(date_pairs)):
            s_date, e_date, path = date_pairs[i]
            if s_date >= frame_start and s_date <= frame_expiry:
                frame_paths.append(path)
                used_indices.add(i)

        if frame_paths:
            dfs = []
            for path in frame_paths:
                df = pd.read_csv(path)

                # Filter: keep only rows where price_problem == False and is_tradable == True
                if "price_problem" in df.columns and "is_tradable" in df.columns:
                    df = df[(df["price_problem"] == False) & (df["is_tradable"] == True)]

                dfs.append(df)

            if dfs:
                frame_df = pd.concat(dfs, axis=0, ignore_index=True)
                frames.append(frame_df)

        # Move to next frame: after expiry
        next_start_date = (datetime.strptime(frame_expiry, "%Y%m%d") + timedelta(days=1)).strftime("%Y%m%d")
        found_next = False
        for i in range(current_index + 1, len(date_pairs)):
            if i in used_indices:
                continue
            if date_pairs[i][0] >= next_start_date:
                current_index = i
                found_next = True
                break

        if not found_next:
            break

    return frames

In [ ]:
def process_expiry_frame(frame_df, cash, lot_s, allocated_lots, hedge_interval_minutes):
    """
    Short an ATM straddle at 9:20 AM on the first day of the expiry cycle.
    Dynamically hedge delta exposure using futures based on Spot price.

    Args:
        frame_df (pd.DataFrame): Data for one expiry cycle (e.g., s_1[i]).
        cash (float): Starting capital.
        lot_s (int): Lot size for the instrument.
        allocated_lots (int): Total quantity to allocate (e.g., 1500 units).
        hedge_interval_minutes (int): Interval in minutes for iteration (e.g., 15).

    Returns:
        dict: Summary including ending cash, trade log, hedge timestamps, and delta log.
    """

    if frame_df.empty:
        print("Empty frame — skipping.")
        return None

    capital = cash
    trade_log = []
    net_futures_held = 0 

    frame_df['Date Time'] = pd.to_datetime(frame_df['Date Time'])
    frame_df['date'] = frame_df['Date Time'].dt.date

    first_date = frame_df['date'].min()
    first_day_df = frame_df[frame_df['date'] == first_date]
    target_time = datetime.strptime("09:20:00", "%H:%M:%S").time()
    # Take snapshot at 9:20 AM on the first day
    snapshot_920 = first_day_df[first_day_df['Date Time'].dt.time == target_time]

    if snapshot_920.empty:
        print("No data at 9:20 AM on the first day — skipping straddle.")
        return None

    spot_price = snapshot_920['Spot'].iloc[0]
    available_strikes = sorted(snapshot_920['Strike'].unique(), key=lambda x: abs(x - spot_price))

    atm_strike = None
    for strike in available_strikes:
        has_ce = not snapshot_920[(snapshot_920['Strike'] == strike) & (snapshot_920['Type'] == 'CE')].empty
        has_pe = not snapshot_920[(snapshot_920['Strike'] == strike) & (snapshot_920['Type'] == 'PE')].empty
        if has_ce and has_pe:
            atm_strike = strike
            break

    if atm_strike is None:
        print("No common strike with both CE and PE found at 9:20 AM — skipping straddle.")
        return None

    atm_ce = snapshot_920[(snapshot_920['Strike'] == atm_strike) & (snapshot_920['Type'] == 'CE')]
    atm_pe = snapshot_920[(snapshot_920['Strike'] == atm_strike) & (snapshot_920['Type'] == 'PE')]

    ce_bid = atm_ce['BidPrice'].iloc[0]
    pe_bid = atm_pe['BidPrice'].iloc[0]

    
    num_lots = int(allocated_lots // lot_s)
    if num_lots == 0:
        print("Allocated quantity too small for even one lot.")
        return None

    premium_received = (ce_bid + pe_bid) * lot_s * num_lots
    capital += premium_received

    trade_time = snapshot_920['Date Time'].iloc[0]
    trade_log.append({
        'action': 'short_straddle',
        'time': trade_time,
        'strike': atm_strike,
        'ce_bid': ce_bid,
        'pe_bid': pe_bid,
        'lots': num_lots,
        'premium_per_lot': ce_bid + pe_bid,
        'total_premium_received': premium_received
    })

    # Generate hedge timestamps
    def generate_hedge_intervals(df, interval_minutes):
        unique_dates = sorted(df['date'].unique())
        hedge_times = []

        start_time = datetime.strptime("09:20:00", "%H:%M:%S").time()
        end_time = datetime.strptime("15:30:00", "%H:%M:%S").time()
        interval = timedelta(minutes=interval_minutes)

        for i, date in enumerate(unique_dates):
            current = datetime.combine(date, start_time)
            day_end = datetime.combine(date, end_time)

            # Adjust for final day: stop before 15:30
            if i == len(unique_dates) - 1:
                day_end -= interval

            while current <= day_end:
                hedge_times.append(current)
                current += interval

        return hedge_times


    hedge_timestamps = generate_hedge_intervals(frame_df, hedge_interval_minutes)

    delta_log = []

    for t in hedge_timestamps:
        snapshot = frame_df[frame_df['Date Time'] == t]
        ce_row = snapshot[(snapshot['Strike'] == atm_strike) & (snapshot['Type'] == 'CE')]
        pe_row = snapshot[(snapshot['Strike'] == atm_strike) & (snapshot['Type'] == 'PE')]

        if not ce_row.empty and not pe_row.empty:
            delta_ce = ce_row['Delta'].iloc[0]
            delta_pe = pe_row['Delta'].iloc[0]
            net_delta = (delta_ce + delta_pe) * lot_s * num_lots
        else:
            net_delta = None

        delta_log.append({
            'time': t,
            'net_delta': net_delta
        })

        if net_delta is not None:
            if abs(net_delta) > lot_s:
                desired_hedge = -net_delta  # we want delta-neutral
                adjustment = desired_hedge - net_futures_held

                # Round adjustment to nearest lot
                hedge_lots = int(abs(adjustment) // lot_s) * lot_s

                if hedge_lots > 0:
                    action = 'long' if adjustment > 0 else 'short'
                    futures_price = snapshot['Spot'].iloc[0]
                    notional = futures_price * hedge_lots

                    # Capital impact
                    if action == 'long':
                        capital -= notional
                        net_futures_held += hedge_lots
                    else:
                        capital += notional
                        net_futures_held -= hedge_lots

                    trade_log.append({
                        'action': f'{action}_futures',
                        'time': t,
                        'hedge_to': desired_hedge,
                        'adjustment': adjustment,
                        'lots': hedge_lots,
                        'price': futures_price,
                        'notional': notional,
                        'net_futures_held_after': net_futures_held
                    })
            else:
                # Delta too small — no hedge
                pass

    # After the hedge loop finishes
    last_date = sorted(frame_df['date'].unique())[-1]
    unwind_time = datetime.combine(last_date, datetime.strptime("15:30:00", "%H:%M:%S").time())

    # Snapshot at 15:30 on last day
    snapshot_1530 = frame_df[frame_df['Date Time'] == unwind_time]

    if snapshot_1530.empty:
        print("No data at 15:30 on last day — cannot unwind positions properly.")
    else:
        # Unwind short straddle: buy CE and PE at AskPrice
        ce_ask = snapshot_1530[(snapshot_1530['Strike'] == atm_strike) & (snapshot_1530['Type'] == 'CE')]['AskPrice']
        pe_ask = snapshot_1530[(snapshot_1530['Strike'] == atm_strike) & (snapshot_1530['Type'] == 'PE')]['AskPrice']

        if ce_ask.empty or pe_ask.empty:
            print("Missing CE or PE ask price at 15:30 on last day — skipping unwind of options.")
        else:
            ce_ask_price = ce_ask.iloc[0]
            pe_ask_price = pe_ask.iloc[0]

            cost_to_close = (ce_ask_price + pe_ask_price) * lot_s * num_lots
            capital -= cost_to_close  # Buying back options costs capital

            trade_log.append({
                'action': 'buyback_straddle',
                'time': unwind_time,
                'strike': atm_strike,
                'ce_ask': ce_ask_price,
                'pe_ask': pe_ask_price,
                'lots': num_lots,
                'total_cost': cost_to_close
            })

        # Unwind futures hedge
        if net_futures_held != 0:
            futures_price = snapshot_1530['Spot'].iloc[0]
            notional = futures_price * abs(net_futures_held)

            if net_futures_held > 0:
                # Sell futures to close long
                capital += notional
                action = 'sell_to_close_futures'
            else:
                # Buy futures to close short
                capital -= notional
                action = 'buy_to_close_futures'

            trade_log.append({
                'action': action,
                'time': unwind_time,
                'lots': abs(net_futures_held),
                'price': futures_price,
                'notional': notional,
                'net_futures_held_after': 0
            })

            net_futures_held = 0  # Reset position
    
    return {
    "starting_cash": cash,
    "ending_cash": capital,
    "trades_executed": len(trade_log),
    "trade_log": trade_log,
    "hedge_timestamps": hedge_timestamps,
    "delta_log": delta_log,
    "final_net_futures_held": net_futures_held  # Should be 0 after unwind
}

In [14]:
stocks = ["ASIANPAINT", "BAJAJ-AUTO"] # Available "ASIANPAINT", "BAJAJ-AUTO", "BAJFINANCE", "HDFCBANK", "ICICIBANK", "RELIANCE", "SBIN", "TATAMOTORS", "TCS", "TITAN"
weight = [1/len(stocks)] * len(stocks) # weights for the assets

# Lot_sizes
df = pd.read_csv('NIFTY_200_Lot_Size.csv')
df['Symbol'] = df['Symbol'].str.strip().str.upper()
df['LOT_SIZE'] = pd.to_numeric(df['LOT_SIZE'], errors='coerce')

lot_sizes = []

for symbol in stocks:
    symbol = symbol.strip().upper()
    lot_size_row = df[df['Symbol'] == symbol]

    if not lot_size_row.empty:
        lot_size = lot_size_row['LOT_SIZE'].values[0]
    else:
        lot_size = None

    lot_sizes.append(lot_size)

In [51]:
s_2 = build_yearly_frames(stocks[1])

In [ ]:
s_2[0]

In [56]:
result = process_expiry_frame(
    frame_df=s_2[0],          
    cash=100000,             
    lot_s=lot_sizes[1],                 
    allocated_lots=500,        # Number of lots allocated for initial short straddle
    hedge_interval_minutes=5 # Hedge interval in minutes (e.g., every 15 minutes)
)

In [ ]:
result

In [29]:
print(result["delta_log"][:5]) 

[{'time': datetime.datetime(2024, 1, 1, 9, 20), 'net_delta': -5.231499083322455}, {'time': datetime.datetime(2024, 1, 1, 9, 25), 'net_delta': 2.2422997196258843}, {'time': datetime.datetime(2024, 1, 1, 9, 30), 'net_delta': 0.031262399033948896}, {'time': datetime.datetime(2024, 1, 1, 9, 35), 'net_delta': -4.274218529758422}, {'time': datetime.datetime(2024, 1, 1, 9, 40), 'net_delta': -6.288580806881328}]


In [32]:
net_deltas = [entry['net_delta'] for entry in result['delta_log']]